In [0]:
from google.colab import drive
drive.mount('/vkcup/')

Drive already mounted at /vkcup/; to attempt to forcibly remount, call drive.mount("/vkcup/", force_remount=True).


In [0]:
!pip install catboost
!pip install xgboost

In [0]:
!pip install transformers

In [0]:
import transformers
import os
import sys
import glob
import torch

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from scipy.sparse import hstack
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import xgboost

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
def fetch_vectors(string_list, batch_size=64):
    # inspired by https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
    DEVICE = torch.device("cuda")
    tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-multilingual-cased")
    model = transformers.BertModel.from_pretrained("bert-base-multilingual-cased")#bert-base-multilingual-cased
    model.to(DEVICE)

    fin_features = []
    for data in chunks(string_list, batch_size):
        tokenized = []
        for x in data:
            x = " ".join(x.strip().split()[:300])
            tok = tokenizer.encode(x, add_special_tokens=True)
            tokenized.append(tok[:512])

        max_len = 512
        padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized])
        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded).to(DEVICE)
        attention_mask = torch.tensor(attention_mask).to(DEVICE)

        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask)

        features = last_hidden_states[0][:, 0, :].cpu().numpy()
        fin_features.append(features)

    fin_features = np.vstack(fin_features)
    return fin_features

In [0]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

In [0]:
train = pd.read_csv("/vkcup/My Drive/Colab Notebooks/vkcup/train.csv", sep=";")
train.head()

,ID,Answer
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0


In [0]:
test = pd.read_csv("/vkcup/My Drive/Colab Notebooks/vkcup/test.csv", sep=";")
test.head()

,ID
0,30001
1,30002
2,30003
3,30004
4,30005


In [0]:
data = pd.read_csv("/vkcup/My Drive/Colab Notebooks/vkcup/data.csv", sep=";")
data.head()

,ID,Question
0,1,Как зовут лодочника на реке Стикс в древнегреч...
1,2,Как в химии обозначается свинец?
2,3,Какой химический элемент преобладает в составе...
3,4,Кто из перечисленных был пажом во времена Екат...
4,5,Когда началась 2 мировая война?


In [0]:
data.shape

(41087, 2)

In [0]:
submission = pd.read_csv("/vkcup/My Drive/Colab Notebooks/vkcup/baseline_solution.csv", sep=",", header=None)
submission.head()

,0,1
0,30001,0.213623
1,30002,0.535675
2,30003,0.041250
3,30004,0.603919
4,30005,0.390166


In [0]:
train_data = data.set_index('ID').join(train.set_index('ID')).dropna()
train_data['Answer'] = train_data['Answer'].astype('int')
train_data.head()

,Question,Answer
ID,,
1,Как зовут лодочника на реке Стикс в древнегреч...,0
2,Как в химии обозначается свинец?,1
3,Какой химический элемент преобладает в составе...,0
4,Кто из перечисленных был пажом во времена Екат...,0
5,Когда началась 2 мировая война?,0


In [0]:
Y = train_data['Answer']
X = pd.DataFrame(train_data['Question'])
X.shape

(30000, 1)

In [0]:
test_data = data.set_index('ID').join(test.set_index('ID'), how='right').dropna()
test_data.head()

,Question
ID,
30001,"Что в литературе обозначает слово ""эпитет""?"
30002,Когда отмечается ДЕНЬ ШОКОЛАДА
30003,Кто из этих бодибилдеров играл Халка
30004,"Последняя на данный момент страна, официально ..."
30005,Как зовут лютоволка Арьи Старк из сериала «Иг...


In [0]:
test_data.shape

(11087, 1)

In [0]:
train_question_body_dense = fetch_vectors(X['Question'])

In [0]:
test_question_body_dense = fetch_vectors(test_data['Question'])

In [0]:
import os
import re
import gc
import pickle  
import random
import keras

import numpy as np
import pandas as pd
import tensorflow as tf
#import tensorflow_hub as hub
from keras import backend as K

from keras.models import Model
from keras.layers import Dense, Input, Dropout, Lambda, GlobalAveragePooling1D, LSTM, Conv1D
from keras.optimizers import Adam
from keras.callbacks import Callback
from scipy.stats import spearmanr, rankdata
from os.path import join as path_join
from numpy.random import seed
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import MultiTaskElasticNet

seed(42)
#tf.random.set_seed(42)
random.seed(42)

Using TensorFlow backend.


In [0]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_question_body_dense, Y, test_size=0.3, random_state=42, shuffle=True)

In [0]:
def create_model():
    inps = Input(shape=(train_question_body_dense.shape[1],))
    x = Dense(512, activation='elu')(inps)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation='elu')(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='elu')(x)
    x = Dropout(1)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inps, outputs=x)
    model.compile(
        optimizer=Adam(lr=1e-4),
        loss=['binary_crossentropy'],
        metrics=[auc]
    )
    model.summary()
    return model

In [0]:
all_predictions = []

model = create_model()
model.fit(X_train, y_train, epochs=50, validation_data=[X_test, y_test], batch_size=32, verbose=True)
all_predictions.append(model.predict(train_question_body_dense))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 768)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               393728    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
__________

In [0]:
metrics.roc_auc_score(Y, all_predictions[0])

0.882766906706491

In [0]:
import re
pred =  model.predict(test_question_body_dense)
ques = ['Кто', 'Что', 'Какой', 'Который', 'Где', 'Когда', 'Почему', 'Зачем', 'Куда', 'Откуда', 'Чей', 'Как', 'Чего','Кем','Чем','Кого','Чего', 'Сколько','Скольких', 'Каких', 'Кому', 'Чему','Чём', 'Чьим', 'Самое','Скольким','Скольким','Самая', 'Самый','Чья','Чью']
ques = ques + [q.lower() for q in ques]

for i, s in enumerate(test_data['Question']):
  if '?' not in s:
    pred[i] = 0
  if s[0].islower():
    pred[i] = 0
  if ' ?' in s:
    pred[i] = 0
  fl = 0
  for q in ques:
    if q in s:
      fl = 1
  if fl ==0:
    pred[i] = 0
  if s.find('   ') != -1:
    pred[i] = 0
  if s.find('\\n') != -1:
    pred[i] = 0
  if s.find(',,') != -1:
    pred[i] = 0
  if s.find(' ,') != -1:
    pred[i] = 0
  if s.find('http') != -1:
    pred[i] = 0
  if s.find('Здравствуйте') != -1:
    pred[i] = 0
  if s.find('? ') != -1:
    pred[i] = 0
  if s.find('!?')!=-1 or s.find('?!')!=-1:
    pred[i] = 0

In [0]:
submission[1] = pred

In [0]:
submission.to_csv('/vkcup/My Drive/Colab Notebooks/vkcup/submission.csv', sep=",", header=None, index=None)